* is applied == 1, value_count
* value_count, user_spec.features

In [1]:
import pandas as pd
import numpy as np
import os
import random
from copy import deepcopy
import math
from itertools import product
import pickle
import joblib
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
user_spec = pd.read_csv('./data/user_spec.csv')

In [3]:
loan_df = pd.read_csv('./data/loan_result.csv')

In [4]:
log_df = pd.read_csv('./data/log_data.csv')

# log 데이터 전처리

In [5]:
log_df = log_df.sort_values(by=['user_id','timestamp'])

In [6]:
log_df = pd.get_dummies(data = log_df, columns=['event'])

In [7]:
result = log_df.groupby(by=['user_id','date_cd']).sum()

In [8]:
result['n_events'] = log_df[['user_id','date_cd','event_CompleteIDCertification']].groupby(by=['user_id','date_cd']).count()

In [9]:
result.reset_index(inplace=True)

In [11]:
result.columns

Index(['user_id', 'date_cd', 'event_CompleteIDCertification',
       'event_EndLoanApply', 'event_GetCreditInfo', 'event_Login',
       'event_OpenApp', 'event_SignUp', 'event_StartLoanApply',
       'event_UseDSRCalc', 'event_UseLoanManage', 'event_UsePrepayCalc',
       'event_ViewLoanApplyIntro', 'n_events'],
      dtype='object')

In [12]:
result = result[['user_id', 'date_cd','n_events',
#                 
'event_SignUp','event_OpenApp', 'event_Login',
'event_ViewLoanApplyIntro', 'event_StartLoanApply',
'event_CompleteIDCertification','event_EndLoanApply',
'event_UseLoanManage','event_UsePrepayCalc','event_UseDSRCalc',
'event_GetCreditInfo'        ]]

In [13]:
result.to_csv('./customData/hj_log_data.csv')

In [30]:
result['date_cd']= pd.to_datetime(user_spec['insert_time']).dt.date

0          2022-05-03
1          2022-06-16
2          2022-05-22
3          2022-05-21
4          2022-03-24
              ...    
2728148    2022-06-29
2728149    2022-03-31
2728150    2022-05-27
2728151    2022-03-14
2728152    2022-05-24
Name: date_cd, Length: 2728153, dtype: object

In [32]:
result

,user_id,date_cd,event_CompleteIDCertification,event_EndLoanApply,event_GetCreditInfo,event_Login,event_OpenApp,event_SignUp,event_StartLoanApply,event_UseDSRCalc,event_UseLoanManage,event_UsePrepayCalc,event_ViewLoanApplyIntro,n_events
0,1,2022-05-03,0,0,2,0,0,0,0,0,0,0,0,2
1,1,2022-06-16,0,0,1,1,0,0,0,0,1,0,0,3
2,7,2022-05-22,0,0,1,0,0,0,0,0,0,0,0,1
3,9,2022-05-21,0,0,3,0,0,0,0,0,0,0,0,3
4,11,2022-03-24,1,1,3,0,1,0,1,0,3,1,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728148,879693,2022-06-29,0,0,0,0,1,0,0,0,0,0,0,1
2728149,879694,2022-03-31,1,3,0,0,0,0,1,0,0,0,1,6
2728150,879695,2022-05-27,1,1,0,0,0,0,1,0,0,0,1,4
2728151,879696,2022-03-14,1,1,3,2,1,0,0,0,2,0,1,11


In [37]:
loan_df

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN
...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN


In [34]:
user_spec

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,date_cd
0,1249046,118218,1985.0,1.0,2022-06-07,660.0,108000000.0,PRIVATEBUSINESS,20151101.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0,2022-06-07
1,954900,553686,1968.0,1.0,2022-06-07,870.0,30000000.0,PRIVATEBUSINESS,20070201.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0,2022-06-07
2,137274,59516,1997.0,1.0,2022-06-07,710.0,30000000.0,FREELANCER,20210901.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0,2022-06-07
3,1570936,167320,1989.0,1.0,2022-06-07,820.0,62000000.0,EARNEDINCOME,20170101.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0,2022-06-07
4,967833,33400,2000.0,1.0,2022-06-07,630.0,36000000.0,EARNEDINCOME,20210901.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0,2022-06-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394211,1864587,489900,2000.0,1.0,2022-03-22,590.0,25000000.0,FREELANCER,202106.0,기타,기타가족소유,5000000.0,사업자금,NaN,NaN,NaN,NaN,2022-03-22
1394212,1327066,151422,1955.0,1.0,2022-03-22,980.0,20000000.0,OTHERINCOME,NaN,기타,자가,50000000.0,생활비,NaN,NaN,1.0,NaN,2022-03-22
1394213,1319606,173524,1983.0,1.0,2022-03-22,750.0,75000000.0,EARNEDINCOME,200908.0,정규직,자가,100000000.0,대환대출,NaN,NaN,8.0,200000000.0,2022-03-22
1394214,1482466,766546,1975.0,1.0,2022-03-22,640.0,50000000.0,EARNEDINCOME,201705.0,정규직,자가,10000000.0,대환대출,NaN,NaN,10.0,117000000.0,2022-03-22


In [20]:
user_spec['insert_time'] = pd.to_datetime(user_spec['insert_time']).dt.date

In [23]:
user_spec['date_cd'] = user_spec['insert_time']

In [35]:
pd.merge(user_spec[['user_id','date_cd','insert_time']], result, on=['date_cd','user_id'])

,user_id,date_cd,insert_time,event_CompleteIDCertification,event_EndLoanApply,event_GetCreditInfo,event_Login,event_OpenApp,event_SignUp,event_StartLoanApply,event_UseDSRCalc,event_UseLoanManage,event_UsePrepayCalc,event_ViewLoanApplyIntro,n_events


In [4]:
log_df.fillna('N', inplace=True)

In [12]:
log_df['mp']=log_df['mp_os'] +'-'+ log_df['mp_app_version']

In [6]:
log_df = pd.get_dummies(data = log_df, columns=['event','mp'])

In [13]:
log_df.drop(['timestamp','mp_os','mp_app_version'],axis=1,inplace=True)

In [ ]:
log_df = log_df.groupby(by=['user_id','date_cd']).sum()

In [10]:
log_df.columns

Index(['user_id', 'timestamp', 'mp_os', 'mp_app_version', 'date_cd',
       'event_CompleteIDCertification', 'event_EndLoanApply',
       'event_GetCreditInfo', 'event_Login', 'event_OpenApp',
       ...
       'mp_iPadOS-3.10.2', 'mp_iPadOS-3.12.0', 'mp_iPadOS-3.12.1',
       'mp_iPadOS-3.13.0', 'mp_iPadOS-3.13.1', 'mp_iPadOS-3.6.0',
       'mp_iPadOS-3.6.1', 'mp_iPadOS-3.7.0', 'mp_iPadOS-3.8.0', 'mp_iPadOS-N'],
      dtype='object', length=325)

In [ ]:
['user_id', 'timestamp'

In [ ]:
log_df = log_df.groupby(by=['user_id','date_cd']).sum()

In [9]:
df= pd.get_dummies(log_df['n_mp'])

In [16]:
df=pd.concat([log_df[['user_id','date_cd']], df],axis=1)

In [ ]:
df = df.groupby(by=['user_id','date_cd']).sum()

In [ ]:
df

In [ ]:
df = df.astype(bool)

In [ ]:
df.sum()

In [14]:
df.astype(bool).T.sum()

11709372    1
11709374    1
2451691     1
2451693     1
7071607     1
           ..
7298671     1
7298672     1
8248077     1
8248080     1
8248082     1
Length: 17843993, dtype: int64

In [13]:
df.T.sum()

11709372    1
11709374    1
2451691     1
2451693     1
7071607     1
           ..
7298671     1
7298672     1
8248077     1
8248080     1
8248082     1
Length: 17843993, dtype: int64

In [15]:
log_df

,user_id,timestamp,mp_os,mp_app_version,date_cd,event_CompleteIDCertification,event_EndLoanApply,event_GetCreditInfo,event_Login,event_OpenApp,event_SignUp,event_StartLoanApply,event_UseDSRCalc,event_UseLoanManage,event_UsePrepayCalc,event_ViewLoanApplyIntro,n_mp
11709372,1,2022-05-03 14:52:28,android,464,2022-05-03,0,0,1,0,0,0,0,0,0,0,0,android-464
11709374,1,2022-05-03 14:52:35,android,464,2022-05-03,0,0,1,0,0,0,0,0,0,0,0,android-464
2451691,1,2022-06-16 23:58:41,Android,3.12.1,2022-06-16,0,0,0,0,0,0,0,0,1,0,0,Android-3.12.1
2451693,1,2022-06-16 23:58:41,Android,3.12.1,2022-06-16,0,0,0,1,0,0,0,0,0,0,0,Android-3.12.1
7071607,1,2022-06-16 23:58:42,android,464,2022-06-16,0,0,1,0,0,0,0,0,0,0,0,android-464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7298671,879696,2022-03-14 05:38:16,Android,3.8.2,2022-03-14,1,0,0,0,0,0,0,0,0,0,0,Android-3.8.2
7298672,879696,2022-03-14 05:42:14,Android,3.8.2,2022-03-14,0,1,0,0,0,0,0,0,0,0,0,Android-3.8.2
8248077,879698,2022-05-24 22:33:24,iOS,N,2022-05-24,0,0,0,0,1,0,0,0,0,0,0,iOS-N
8248080,879698,2022-05-24 22:33:32,iOS,N,2022-05-24,0,0,0,0,0,0,1,0,0,0,0,iOS-N


In [24]:
log_df = pd.get_dummies(data = log_df, columns=['n_mp'])

In [ ]:
log_df=pd.concat([log_df.loc[:, log_df.columns[:16]], log_df.loc[:, log_df.columns[16:]]!=0], axis=1)

In [ ]:
log_df

In [ ]:
log_df.groupby(by=['user_id','date_cd']).sum()